<a href="https://colab.research.google.com/github/BamByeol/learning-ML/blob/HW220129/HW0129.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 소프트맥스 회귀의 비용함수 구하기 05.03
import torch
import torch.nn.functional as F
torch.manual_seed(1)


#low-level
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim=0) #dim 의미가...정확히 먼지 모르겠넹...0이랑 -1이랑 차이가 없음.............ㅠㅠ
print(hypothesis)
#result- 3개의 원소값이 0과1사이의 벡터값으로 바뀜.
#tensor([0.0900, 0.2447, 0.6652]) 

hypothesis.sum() #벡터의 합이 1이 되는지 확인
#result
#tensor(1.)

#cost-function
z = torch.rand(3, 5, requires_grad=True) # 임의의 3X5 행렬

hypothesis = F.softmax(z, dim=1) #각 샘플에 대해서 소프트맥스 함수를 적용하여야 하므로 두번째 차원에 대해서 소프트맥스 함수를 적용한다는 의미에서 dim=1을 써준다는데 dim의 범위는 왜 [-2,1]일까
print(hypothesis) #소프트맥스 함수의 출력값은 결국 예측값
#result
#tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
#        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
#        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)

#dim=0 result
#tensor([[0.3402, 0.3401, 0.3090, 0.5217, 0.3453],
#       [0.2954, 0.2338, 0.2847, 0.2716, 0.2518],
#       [0.3645, 0.4261, 0.4063, 0.2067, 0.4029]], grad_fn=<SoftmaxBackward0>)

#dim=2 result error : Dimension out of range (expected to be in range of [-2, 1], but got 2)

#dim=-1 result
#tensor([[0.2570, 0.1524, 0.2303, 0.1721, 0.1882],
#        [0.1178, 0.1501, 0.2499, 0.1687, 0.3135],
#        [0.2615, 0.1711, 0.1254, 0.2375, 0.2045]], grad_fn=<SoftmaxBackward0>)

#dim=-2 result
#tensor([[0.2447, 0.4240, 0.3654, 0.4763, 0.2867],
#        [0.3114, 0.3035, 0.2902, 0.2092, 0.2661],
#        [0.4438, 0.2725, 0.3444, 0.3145, 0.4472]], grad_fn=<SoftmaxBackward0>)

#샘플에 대한 임의의 레이블 생성
y = torch.randint(5, (3,)).long()
print(y)
#result
#tensor([0, 2, 1])

#각 레이블에 대해서 원-핫 인코딩 - 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) #모든 원소가 0의 값을 가진 3 × 5 텐서를 만듦 / 텐서는 y_one_hot에 저장이 된 상태
y_one_hot.scatter_(1, y.unsqueeze(1), 1) #y.unsqueeze(1)를 하면 (3,)의 크기를 가졌던 y 텐서는 (3 × 1) 텐서가 됨
                                         # scatter의 첫번째 인자로 dim=1에 대해서 수행하라고 알려줌
                                         #세번째 인자에 숫자 1을 넣어주므로서 두번째 인자인 y_unsqeeze(1)이 알려주는 위치에 숫자 1을 넣도록 함
# 연산 뒤에 _를 붙이면 In-place Operation (덮어쓰기 연산)임 (텐서 조작하기 2챕터에서)

#print(y.unsqueeze(1))
#result
#tensor([[0],
        #[2],
        #[1]])

#print(y_one_hot)
#tensor([[1., 0., 0., 0., 0.],
        #[0., 0., 1., 0., 0.],
        #[0., 1., 0., 0., 0.]])

cost_low = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
#        = (y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()  // F.softmax() + torch.log() = F.log_softmax() 
print(cost_low)
#result
#tensor(1.4689, grad_fn=<MeanBackward0>)

print("\n--------------------------------------------------------------------------------------------\n")


#high-level
F.log_softmax(z, dim=1) #F.softmax() + torch.log() = F.log_softmax()
cost_high = F.nll_loss(F.log_softmax(z, dim=1),y)  # F.nll_loss()를 사용할 때는 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용
                                                # nll이란 Negative Log Likelihood의 약자입니다. 위에서 nll_loss는 F.log_softmax()를 수행한 후에 남은 수식들을 수행한다는 뜻
        # = F.cross_entropy(z, y) // F.log_softmax() + F.nll_loss() = F.cross_entropy()

print(cost_high)     
#result
#tensor(1.4689, grad_fn=<NllLossBackward0>)

#low와 high의 차이를 발견했다
#grad fn = <MeanBackward0> 과 grad_fn=<NllLossBackward0>)


In [ ]:
#소프트맥스 회귀 구현 - low level
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

#train_data_set
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] #4개특성8개샘플
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #3개의클래스
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

#클래스의 개수는 3개이므로 y_train에 원-핫 인코딩한 결과는 8 × 3의 개수를 가져야 함
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True) #y_train에서 원-핫 인코딩을 한 결과인 y_one_hot의 크기는 8 × 3입니다. 즉, W 행렬의 크기는 4 × 3이어야 함
b = torch.zeros(1, requires_grad=True)
# optimizer 설정 - 경사하강법
optimizer = optim.SGD([W, b], lr=0.1)

# F.softmax()와 torch.log()를 사용하여 가설과 비용 함수를 정의
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

#result
#Epoch    0/1000 Cost: 1.098612
#Epoch  100/1000 Cost: 0.761050
#Epoch  200/1000 Cost: 0.689991
#Epoch  300/1000 Cost: 0.643229
#Epoch  400/1000 Cost: 0.604117
#Epoch  500/1000 Cost: 0.568255
#Epoch  600/1000 Cost: 0.533922
#Epoch  700/1000 Cost: 0.500291
#Epoch  800/1000 Cost: 0.466908
#Epoch  900/1000 Cost: 0.433507
#Epoch 1000/1000 Cost: 0.399962

In [ ]:
#소프트맥스 회귀 구현 - high level
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

#train_data_set
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] #4개특성8개샘플
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #3개의클래스
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

# F.cross_entropy()를 사용하여 비용 함수를 구현 / F.cross_entropy()는 그 자체로 소프트맥스 함수를 포함하고 있으므로 가설에서는 소프트맥스 함수를 사용할 필요가 없음.
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

#result
#Epoch    0/1000 Cost: 1.098612
#Epoch  100/1000 Cost: 0.761050
#Epoch  200/1000 Cost: 0.689991
#Epoch  300/1000 Cost: 0.643229
#Epoch  400/1000 Cost: 0.604117
#Epoch  500/1000 Cost: 0.568256
#Epoch  600/1000 Cost: 0.533922
#Epoch  700/1000 Cost: 0.500291
#Epoch  800/1000 Cost: 0.466908
#Epoch  900/1000 Cost: 0.433507
#Epoch 1000/1000 Cost: 0.399962

In [ ]:
#소프트맥스 회귀 구현 - nn.module
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

#train_data_set
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] #4개특성8개샘플
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #3개의클래스
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3) #  output_dim이 1이었던 선형 회귀때와 달리 output_dim은 이제 클래스의 개수여야 함

# F.cross_entropy()를 사용할 것이므로 따로 소프트맥스 함수를 가설에 정의하지 않음
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

#result
#Epoch    0/1000 Cost: 1.616785
#Epoch  100/1000 Cost: 0.658891
#Epoch  200/1000 Cost: 0.573443
#Epoch  300/1000 Cost: 0.518151
#Epoch  400/1000 Cost: 0.473265
#Epoch  500/1000 Cost: 0.433516
#Epoch  600/1000 Cost: 0.396563
#Epoch  700/1000 Cost: 0.360914
#Epoch  800/1000 Cost: 0.325392
#Epoch  900/1000 Cost: 0.289178
#Epoch 1000/1000 Cost: 0.254148        

In [ ]:
#소프트맥스 회귀 구현 - 클래스 사용
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

#train_data_set
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] #4개특성8개샘플
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #3개의클래스
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

#result
#Epoch    0/1000 Cost: 1.616785
#Epoch  100/1000 Cost: 0.658891
#Epoch  200/1000 Cost: 0.573443
#Epoch  300/1000 Cost: 0.518151
#Epoch  400/1000 Cost: 0.473265
#Epoch  500/1000 Cost: 0.433516
#Epoch  600/1000 Cost: 0.396563
#Epoch  700/1000 Cost: 0.360914
#Epoch  800/1000 Cost: 0.325392
#Epoch  900/1000 Cost: 0.289178
#Epoch 1000/1000 Cost: 0.254148

In [ ]:
# Q.왜 cost 값이 low = high인지 / nn.module = class인지 그리고 왜 앞서 두개와 뒤의 두개의 cost값은 서로 다른지...